import ipotrant libraries

In [1]:
import ROOT
import math
from array import array
import numpy as np
from ROOT import gROOT, AddressOf

Welcome to JupyROOT 6.29/01


Set common variables

In [2]:
read_ampt_dat_files = True

ampt_files_path = "/home/yoren/bnl/AMPT/Artem_heau/output_taxi/"

N_files = 200

Define structures

In [3]:
gROOT.ProcessLine(
"struct event {\
  Int_t           number;\
  Int_t         ncharge_y;\
  Float_t         phi_bbc;\
  Float_t         phi_cnt;\
  Float_t         phi_fvtx;\
  };"
); 


class event:
  
  def __init__(self, input_array):
    self.number = int(input_array[0])-101
    self.check = int(input_array[1])
    self.nch_tot = int(input_array[2])
    self.bimp = float(input_array[3])
    self.npart1 = int(input_array[4])
    self.npart2 = int(input_array[5])
    self.ncoll_ell = int(input_array[6])
    self.ncoll = int(input_array[7])
    self.ncoll1 = int(input_array[8])
    self.ncharge_y = int(input_array[9])
    self.phi_bbc = float(input_array[10])
    self.phi_cnt = float(input_array[11])
    self.phi_fvtx = float(input_array[12])
  
  
  def __str__(self):
    return f"number = {self.number}, check = {self.check}, nch_tot = {self.nch_tot}, bimp = {self.bimp}, npart1 = {self.npart1}, npart2 = {self.npart2}, "+f"ncoll_ell = {self.ncoll_ell}, ncoll = {self.ncoll}, ncoll1 = {self.ncoll1}, ncharge_y = {self.ncharge_y}, phi_bbc = {self.phi_bbc}, phi_cnt = {self.phi_cnt}, phi_fvtx = {self.phi_fvtx},"
  
  def get_values(self):
    list = [self.number, self.ncharge_y, self.phi_bbc, self.phi_cnt, self.phi_fvtx]
    for i in range(len(list)):
      if list[i]>1e6 or list[i]<-5:
        list[i]=9999
    #arr = np.array(list)
    return list
  
  def set_event(self, mystruck):
    mystruck.number = self.number
    mystruck.ncharge_y = self.ncharge_y
    mystruck.phi_bbc = self.phi_bbc
    mystruck.phi_cnt = self.phi_cnt
    mystruck.phi_fvtx = self.phi_fvtx

  
class particles:
  def __init__(self):
    self.px_pi0 = []
    self.py_pi0 = []
    self.pz_pi0 = []
    self.m_pi0 = []
    self.px = []
    self.py = []
    self.pz = []
    self.m = []

  def __str__(self):
    return f"N_part = {len(self.m), self.m}"

  def add_charged_particle(self, input_mom):
    self.px.append(float(input_mom[0]))
    self.py.append(float(input_mom[1]))
    self.pz.append(float(input_mom[2]))
    self.m.append(float(input_mom[3]))

  def add_pi0(self, input_mom):
    self.px_pi0.append(float(input_mom[0]))
    self.py_pi0.append(float(input_mom[1]))
    self.pz_pi0.append(float(input_mom[2]))
    self.m_pi0.append(float(input_mom[3]))

Create TTree creator

In [4]:
branch_input = [array('i', [0]), array('i', [0]), array('d', [0]), array('d', [0]), array('d', [0])]
N = 100
e_input = [np.array([0.]*N),np.array([0.]*N),np.array([0.]*N),np.array([0.]*N)]
pi0_input = [np.array([0.]*N),np.array([0.]*N),np.array([0.]*N),np.array([0.]*N)]
def ttree_creator(name):
    tree = ROOT.TTree(name,name)
    tree.Branch('number', branch_input[0], 'number/I')
    tree.Branch('ncharge_y', branch_input[1], 'ncharge_y/I')
    tree.Branch('phi_bbc', branch_input[2], 'phi_bbc/D')
    tree.Branch('phi_cnt', branch_input[3], 'phi_cnt/D')
    tree.Branch('phi_fvtx', branch_input[4], 'phi_fvtx/D')

    tree.Branch("px", e_input[0], 'px[' + str(N) + ']/D')
    tree.Branch("py", e_input[1], 'py[' + str(N) + ']/D')
    tree.Branch("pz", e_input[2], 'pz[' + str(N) + ']/D')
    tree.Branch("m",  e_input[3],  'm[' + str(N) + ']/D')

    tree.Branch("px_pi0", pi0_input[0], 'px_pi0[' + str(N) + ']/D')
    tree.Branch("py_pi0", pi0_input[1], 'py_pi0[' + str(N) + ']/D')
    tree.Branch("pz_pi0", pi0_input[2], 'pz_pi0[' + str(N) + ']/D')
    tree.Branch("m_pi0",  pi0_input[3],  'm_pi0[' + str(N) + ']/D')
    
    return tree



    #mystruck= ROOT.event()
    #tree.Branch("event",mystruck,"number/I:ncharge_y/I:phi_bbc/F:phi_cnt/F:phi_fvtx/F")
    #N=5
    #npa = np.array(N*[ 0.2 ])
    #tree.Branch('nparrayb', npa, 'nparrayb[' + str(N) + ']/D')
    #tree.Branch("point",event,"number:check:nch_tot/i:bimp/F:npart1:npart2:ncoll_ell:ncoll:ncoll1:ncharge_y/I:phi_bbc:phi_cnt:phi_fvtx/F")


Create file reader

In [5]:
def file_reader(filename, tree):
    f = open(filename, "r")
    part = particles()
    for line in f:
        line = line.split()
        if float(line[0]) > 100:
            if float(line[0]) > 101:
                if len(part.px)>99:
                    print(part)
                for i in range(len(part.px)):
                    e_input[0][i]=part.px[i]
                    e_input[1][i]=part.py[i]
                    e_input[2][i]=part.pz[i]
                    e_input[3][i]=part.m[i]
                for i in range(len(part.px),100):
                    e_input[0][i]=0
                    e_input[1][i]=0
                    e_input[2][i]=0
                    e_input[3][i]=0
                for i in range(len(part.px_pi0)):
                    pi0_input[0][i]=part.px_pi0[i]
                    pi0_input[1][i]=part.py_pi0[i]
                    pi0_input[2][i]=part.pz_pi0[i]
                    pi0_input[3][i]=part.m_pi0[i]
                for i in range(len(part.px_pi0),100):
                    pi0_input[0][i]=0
                    pi0_input[1][i]=0
                    pi0_input[2][i]=0
                    pi0_input[3][i]=0
                tree.Fill()
                #e_input[0], e_input[1], e_input[2], e_input[3] = np.array([0.]*N), np.array([0.]*N), np.array([0.]*N), np.array([0.]*N)
            test = event(line)
            npa = test.get_values()
            for i in range(5):
                branch_input[i][0] = npa[i]            
            part = particles()
        else :
            if float(line[3]) == 0.1350: part.add_pi0(line)
            else: part.add_charged_particle(line)
    
    #for i in range(len(part.px)):
        #e_input[0][i]=part.px[i]
    #tree.Fill()


Read AMPT files and create TTree

In [6]:
tree = ttree_creator("yolo")
if read_ampt_dat_files:  
    for i in range(1,1+N_files):
        filename = ampt_files_path+f"ampt_{i}.dat"
        file_reader(filename, tree)
tree.Print()

Write Tree to RootFile

In [7]:
if read_ampt_dat_files:
  
  output_file = ROOT.TFile.Open("HeAu_Artem.root","RECREATE")
  tree.Write()
  output_file.Close()

Read TTree

In [12]:
inFile = ROOT.TFile.Open('HeAu_Artem.root')
tree_read = inFile.Get("yolo")


#for ev in tree_read:
    #for i in range(len(ev.m)):
        #if ev.m[i] == 0.0005:
            #print("electron")

inFile.Close()


electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
electron
e

KeyboardInterrupt: 

In [9]:
import math

def angle(vector1, vector2):
    x1, y1, z1 = vector1[1], vector1[2], vector1[3]
    x2, y2, z2 = vector2[1], vector2[2], vector2[3]
    inner_product = x1*x2 + y1*y2 + z1*z2
    len1 = math.hypot(x1, y1, z1)
    len2 = math.hypot(x2, y2, z2)
    #print(inner_product,len1,len2)
    rel = inner_product/(len1*len2)
    if abs(rel)>1:
        rel = 1
    return math.acos(rel)

veto_angle = math.atan(20./522)
print(veto_angle)

0.03829544464261488


In [10]:
def inv_mass(vector1, vector2):
    e1, px1, py1, pz1 = vector1[0], vector1[1], vector1[2], vector1[3]
    e2, px2, py2, pz2 = vector2[0], vector2[1], vector2[2], vector2[3]
    m2 = (e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2
    if m2<0: m2=0
    return math.sqrt(m2)

In [11]:
#second(156,5000)
import multiprocess as mp

Ntr = nThread
Nev = nEvents
seed0 = 15156
seed_iter = 43324

pn_ids = [int(2212), int(2112)]

projectile_params = [pn_ids[int(1.34*i/(Ntr))] for i in range(Ntr)]
target_params = [pn_ids[min(int(4*i/Ntr), 1)] for i in range(Ntr)]
print(projectile_params, target_params)

pool = mp.Pool(Ntr)
hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, 2212, 2212) for i in range(Ntr)])
#hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, 1000020040, 1000791970) for i in range(Ntr)]) #1000020030
#hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, projectile_params[i], target_params[i]) for i in range(Ntr)])
pool.close()

NameError: name 'nThread' is not defined

In [ ]:
hist_Nch = hist_array[0][0]
for i in range(1, Ntr):
    hist_Nch.Add(hist_array[i][0])
hist_pt = hist_array[0][1]
for i in range(1, Ntr):
    hist_pt.Add(hist_array[i][1])
hist_imass = hist_array[0][2]
for i in range(1, Ntr):
    hist_imass.Add(hist_array[i][2])
hist_Nch_mid = hist_array[0][3]
for i in range(1, Ntr):
    hist_Nch_mid.Add(hist_array[i][3])

NameError: name 'hist_array' is not defined